In [1]:
#import
from qiskit import QuantumCircuit, Aer, QuantumRegister, ClassicalRegister, execute
import numpy as np
from qiskit.circuit.library import QFT

# convert number into binary numbers
def set_input_state(a,b,t):
    
    # I have chossen not 4 bits as it overflows when addition crosses 15
    get_binary = lambda x : '{0:{fill}6b}'.format(x, fill='0')
    
    a_binary = get_binary(a)
    b_binary = get_binary(b)
    t_binary = get_binary(t)
    
    max_len = len(a_binary)
    max_len1 = len(t_binary)

    r_a = QuantumRegister(max_len,  'a')
    r_b = QuantumRegister(max_len,  'b')
    r_t = QuantumRegister(max_len1,  't')
    anc = QuantumRegister(max_len, 'anc')
    flag = QuantumRegister(1, 'f')

    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(r_a, r_b, r_t, cr,anc,flag)
    
    for i in range(max_len):
        if a_binary[i] == '1':
            qc.x(r_a[max_len-1-i])
        if b_binary[i] == '1':
            qc.x(r_b[max_len-1-i])
            
    for i in range(max_len1):  
        if t_binary[i] == '1':
            qc.x(r_t[max_len-1-i])
            
    return qc, r_a, r_b, r_t, cr, max_len, max_len1, anc, flag


# phases to be applied
def calcPhase(control_q, target_q, num_qubits):
    
    k = num_qubits - target_q
    phase = ((2 * np.pi) * (2 ** control_q)) / (2 ** k)
    
    return phase


def find_the_primes_numbers (target: int, list_number):
    for a in list_number:
        for b in list_number:
            qc, r_a, r_b, r_t, cr, max_len, max_len1, anc, flag = set_input_state(a, b, target)
            
            #qc.barrier()
            n = max_len
            # applying QFT
            qc.append(QFT(n), r_a)
            for i in range(0,n):
                for j in range(n - i):
                    phase = calcPhase(control_q = i, target_q = j, num_qubits = n)
                    if phase == 2 * np.pi:
                        break
                    qc.cp(phase, r_b[i], r_a[j])
                qc.barrier()
            
            qc.append(QFT(n).inverse(), r_a)
        
            qc.barrier()
            
            #oracle for comparison 
            for i in range(max_len):
                qc.cx(r_a[i], anc[i])
                qc.cx(r_t[i], anc[i])
                qc.barrier()
            qc.x(anc)
            qc.mcx(anc, flag)
            
            #display(qc.draw('mpl'))
            
           
            qc.measure(flag, cr)

                    
            #display(qc.draw('mpl'))
            backend = Aer.get_backend('qasm_simulator')
            job = execute(qc, backend, shots = 100)
            result = job.result()
            counts = result.get_counts(qc)
            # using the oracle for comparing target with sum 
            if int(max(counts, key=counts.get), 2) == 1:
                return a, b
            #print(counts)  

In [2]:
A = find_the_primes_numbers  (18,[1,3,5,7,11,13,15])
print(A)

(3, 15)
